#  **Model 1: Baseline Linear Pricing**
Implementation of Pricing logic and Bokeh Visualisation

**Baseline Linear Pricing**

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")

In [3]:
# Combining 'LastUpdatedDate' and 'LastUpdatedTime' into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

In [4]:
# Sorting data by parking lot and time
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [5]:
# Defining base price and alpha (tunable)
base_price = 10         # Starting price in dollars
alpha = 2.5             # Controls how fast price increases

In [6]:
# Calculating baseline linear price for each parking lot
baseline_prices = []

In [7]:
# Grouping data by each parking lot
for lot_id, group in df.groupby('SystemCodeNumber'):
    group = group.copy()               # Work on a copy of the group
    current_price = base_price         # Start with base price for each lot
    prices = []                        # Store prices for this group

    for _, row in group.iterrows():
        occ_ratio = row['Occupancy'] / row['Capacity']      # Occupancy percentage
        new_price = current_price + alpha * occ_ratio        # Linear price formula
        prices.append(new_price)
        current_price = new_price                            # Update price for next step

    baseline_prices.extend(prices)  # Add this group's prices to the full list

In [8]:
# Adding the computed price column to the dataframe
df['BaselinePrice'] = baseline_prices

In [9]:
# Checking the result
print(df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'BaselinePrice']].head(10))

  SystemCodeNumber           Timestamp  Occupancy  Capacity  BaselinePrice
0      BHMBCCMKT01 2016-10-04 07:59:00         61       577      10.264298
1      BHMBCCMKT01 2016-10-04 08:25:00         64       577      10.541594
2      BHMBCCMKT01 2016-10-04 08:59:00         80       577      10.888215
3      BHMBCCMKT01 2016-10-04 09:32:00        107       577      11.351820
4      BHMBCCMKT01 2016-10-04 09:59:00        150       577      12.001733
5      BHMBCCMKT01 2016-10-04 10:26:00        177       577      12.768631
6      BHMBCCMKT01 2016-10-04 10:59:00        219       577      13.717504
7      BHMBCCMKT01 2016-10-04 11:25:00        247       577      14.787695
8      BHMBCCMKT01 2016-10-04 11:59:00        259       577      15.909879
9      BHMBCCMKT01 2016-10-04 12:29:00        266       577      17.062392


**Bokeh Visualization (Single Lot)**

In [10]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.palettes import Category10
from bokeh.palettes import Category20

In [11]:
# Enabling Bokeh output inside Colab
output_notebook()

In [12]:
# Filtering data for one parking lot (e.g., BHMBCCMKT01)
sample_lot = "BHMBCCMKT01"
df_sample = df[df['SystemCodeNumber'] == sample_lot]

In [13]:
# Creating a Bokeh Figure
p = figure(title=f"Baseline Pricing Over Time - {sample_lot}",
           x_axis_label="Time",
           y_axis_label="Price ($)",
           x_axis_type='datetime',
           width=800,
           height=400)

In [14]:
# Plotting the line
p.line(df_sample['Timestamp'], df_sample['BaselinePrice'],
       line_width=2, color=Category10[10][0], legend_label=sample_lot)

GlyphRenderer(id='p1058', ...)

In [15]:
# Addition of hover tool for visualisation
hover = HoverTool(tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y{$0.00}")],
                  formatters={"@x": "datetime"},
                  mode="vline")
p.add_tools(hover)

In [16]:
p.legend.location = "top_left"
p.title.text_font_size = '16pt'

In [17]:
show(p)

**Bokeh Visualisation of All 14 Parking Lots**

In [18]:
from bokeh.palettes import Category20

p_all = figure(title="Model 1: Baseline Price for All Parking Lots",
               x_axis_label="Time",
               y_axis_label="Price ($)",
               x_axis_type='datetime',
               width=950,
               height=500)

# Picking color palette (at least 14 colors)
colors = Category20[20]
lots = df['SystemCodeNumber'].unique()

# Loop through each lot and plot its BaselinePrice line
for i, lot in enumerate(lots):
    df_lot = df[df['SystemCodeNumber'] == lot]
    p_all.line(df_lot['Timestamp'], df_lot['BaselinePrice'],
               line_width=2,
               color=colors[i % len(colors)],
               legend_label=lot)

# Style the chart
p_all.title.text_font_size = '16pt'
p_all.legend.location = "top_right"
p_all.legend.click_policy = "hide"  # Let user hide/show lines

# Show the plot
show(p_all)

# **Model 2: Demand-Based Dynamic Pricing**
Creating a smarter pricing model that adjusts based on demand driven by multiple features — not just occupancy.

**Demand-Based Dynamic Pricing**

In [19]:
# Encoding categorical variables
traffic_map = {'low': 0, 'medium': 0.5, 'high': 1}
df['TrafficEncoded'] = df['TrafficConditionNearby'].map(traffic_map)

vehicle_map = {'car': 1, 'bike': 0.6, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map).fillna(1)

In [20]:
# Defining weights for demand function
alpha = 3      # for occupancy ratio
beta = 0.5     # for queue length
gamma = 1.5    # for special day
delta = 1.2    # for traffic level
epsilon = 1.0  # for vehicle weight

In [21]:
# Computing raw demand score for each row
df['DemandScore'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] +
    gamma * df['IsSpecialDay'] +
    delta * df['TrafficEncoded'] +
    epsilon * df['VehicleWeight']
)

In [22]:
# Normalising demand score between 0 and 1
d_min = df['DemandScore'].min()
d_max = df['DemandScore'].max()

df['NormalizedDemand'] = (df['DemandScore'] - d_min) / (d_max - d_min)

In [23]:
# Calculation of Final Dynamic Price
base_price = 10
lambda_ = 0.5  # how sensitive price is to demand

df['DemandPrice'] = base_price * (1 + lambda_ * df['NormalizedDemand'])

In [24]:
# Clipping the prices between $5 and $20
df['DemandPrice'] = df['DemandPrice'].clip(lower=5, upper=20)

In [25]:
# Checking result
print(df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficConditionNearby',
          'VehicleType', 'DemandScore', 'NormalizedDemand', 'DemandPrice']].head(10))

  SystemCodeNumber           Timestamp  Occupancy  QueueLength  \
0      BHMBCCMKT01 2016-10-04 07:59:00         61            1   
1      BHMBCCMKT01 2016-10-04 08:25:00         64            1   
2      BHMBCCMKT01 2016-10-04 08:59:00         80            2   
3      BHMBCCMKT01 2016-10-04 09:32:00        107            2   
4      BHMBCCMKT01 2016-10-04 09:59:00        150            2   
5      BHMBCCMKT01 2016-10-04 10:26:00        177            3   
6      BHMBCCMKT01 2016-10-04 10:59:00        219            6   
7      BHMBCCMKT01 2016-10-04 11:25:00        247            5   
8      BHMBCCMKT01 2016-10-04 11:59:00        259            5   
9      BHMBCCMKT01 2016-10-04 12:29:00        266            8   

  TrafficConditionNearby VehicleType  DemandScore  NormalizedDemand  \
0                    low         car     1.817158          0.087811   
1                    low         car     1.832756          0.088992   
2                    low         car     2.415945          0

**Bokeh Visualization (Single Lot)**

In [26]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.palettes import Category10

In [27]:
output_notebook()

In [28]:
sample_lot = "BHMBCCMKT01"  # This can be changed to any of the 14 IDs
df_sample = df[df['SystemCodeNumber'] == sample_lot]

In [29]:
p = figure(title=f"Model 2: Demand-Based Pricing Over Time – {sample_lot}",
           x_axis_label="Time",
           y_axis_label="Price ($)",
           x_axis_type='datetime',
           width=800,
           height=400)

In [30]:
p.line(df_sample['Timestamp'], df_sample['DemandPrice'],
       line_width=2, color=Category10[10][1], legend_label="Demand-Based Price")

GlyphRenderer(id='p1341', ...)

In [31]:
hover = HoverTool(tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y{$0.00}")],
                  formatters={"@x": "datetime"},
                  mode="vline")
p.add_tools(hover)

In [32]:
p.legend.location = "top_left"
p.title.text_font_size = '16pt'

show(p)

In [33]:
from bokeh.palettes import Category20

# Display Bokeh plots inside notebook
output_notebook()

# Create the figure
p_all_model2 = figure(title="Model 2: Demand-Based Price for All Parking Lots",
                      x_axis_label="Time",
                      y_axis_label="Price ($)",
                      x_axis_type='datetime',
                      width=950,
                      height=500)

# Use Category20 palette for distinct colors
colors = Category20[20]
lots = df['SystemCodeNumber'].unique()

# Loop through each lot and plot its DemandPrice
for i, lot in enumerate(lots):
    df_lot = df[df['SystemCodeNumber'] == lot]
    p_all_model2.line(df_lot['Timestamp'], df_lot['DemandPrice'],
                      line_width=2,
                      color=colors[i % len(colors)],
                      legend_label=lot)

# Style the chart
p_all_model2.title.text_font_size = '16pt'
p_all_model2.legend.location = "top_right"
p_all_model2.legend.click_policy = "hide"

# Show the plot
show(p_all_model2)

# **Model 3 — Competitive Pricing**
Using latitude & longitude to compute distances between parking lots.
Adjusting price dynamically based on neighbor prices.

 **Implementing Competitive Pricing Logic**

In [34]:
import numpy as np

lot_locations = df[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Haversine function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = phi2 - phi1
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# Creating distance matrix
distance_matrix = pd.DataFrame(index=lot_locations['SystemCodeNumber'],
                               columns=lot_locations['SystemCodeNumber'],
                               dtype=float)

for i, row1 in lot_locations.iterrows():
    for j, row2 in lot_locations.iterrows():
        d = haversine(row1['Latitude'], row1['Longitude'], row2['Latitude'], row2['Longitude'])
        distance_matrix.loc[row1['SystemCodeNumber'], row2['SystemCodeNumber']] = d


In [35]:
# Defining radius for "nearby" lots
NEARBY_RADIUS_KM = 1.0

# Creating a dictionary to store nearby lots for each lot
nearby_lots_map = {}

for lot in distance_matrix.index:
    nearby = distance_matrix.loc[lot][(distance_matrix.loc[lot] > 0) &
                                      (distance_matrix.loc[lot] <= NEARBY_RADIUS_KM)].index.tolist()
    nearby_lots_map[lot] = nearby

# Viewing the first 5 mappings
for k, v in list(nearby_lots_map.items())[:5]:
    print(f"{k} → {v}")


BHMBCCMKT01 → ['BHMBCCTHL01', 'BHMEURBRD01', 'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98', 'Shopping']
BHMBCCTHL01 → ['BHMBCCMKT01', 'BHMEURBRD01', 'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98', 'Shopping']
BHMEURBRD01 → ['BHMBCCMKT01', 'BHMBCCTHL01', 'Shopping']
BHMMBMMBX01 → []
BHMNCPHST01 → ['BHMBCCMKT01', 'BHMBCCTHL01', 'BHMNCPNST01', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98']


In [36]:
# Creating a CompetitivePrice column based on Model 2 demand prices
df = df.sort_values(['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Setting occupancy threshold
threshold = 0.9  # 90%

# Function to adjust price based on competition
def adjust_price(row):
    lot = row['SystemCodeNumber']
    time = row['Timestamp']
    base_price = row['DemandPrice']

    # Getting occupancy ratio
    occ_ratio = row['Occupancy'] / row['Capacity']

    # Getting nearby lots
    nearby_lots = nearby_lots_map.get(lot, [])

    # If no competitors, return original price
    if not nearby_lots:
        return base_price

    # Getting prices of nearby lots at the same timestamp
    nearby_prices = df[(df['SystemCodeNumber'].isin(nearby_lots)) & (df['Timestamp'] == time)]['DemandPrice']

    if nearby_prices.empty:
        return base_price

    avg_nearby_price = nearby_prices.mean()

    # Logic
    if occ_ratio > threshold and avg_nearby_price < base_price:
        return max(base_price - 1.0, 5)  # Reduce price slightly, not below $5
    elif avg_nearby_price > base_price:
        return min(base_price + 0.5, 20)  # Increase slightly, not above $20
    else:
        return base_price

# Applying the function
df['CompetitivePrice'] = df.apply(adjust_price, axis=1)

# Checking results
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'DemandPrice', 'CompetitivePrice']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,Capacity,DemandPrice,CompetitivePrice
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.439054,10.939054
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,10.444958,10.944958
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,10.665699,11.165699
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,10.718834,11.218834
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,10.652054,11.152054
5,BHMBCCMKT01,2016-10-04 10:26:00,177,577,11.045845,11.545845
6,BHMBCCMKT01,2016-10-04 10:59:00,219,577,12.339720,12.839720
7,BHMBCCMKT01,2016-10-04 11:25:00,247,577,NaN,NaN
8,BHMBCCMKT01,2016-10-04 11:59:00,259,577,NaN,NaN
9,BHMBCCMKT01,2016-10-04 12:29:00,266,577,12.470066,12.970066


**Bokeh Visualisation of Model 2 vs Model 3 (Single Lot)**

In [37]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.palettes import Category10

output_notebook()

sample_lot = "BHMBCCMKT01"
df_sample = df[df['SystemCodeNumber'] == sample_lot]

p = figure(title=f"Pricing Comparison: Model 2 vs Model 3 – {sample_lot}",
           x_axis_label="Time",
           y_axis_label="Price ($)",
           x_axis_type='datetime',
           width=900,
           height=450)

# Plotting Model 2 price
p.line(df_sample['Timestamp'], df_sample['DemandPrice'],
       line_width=2, color=Category10[10][0], legend_label="Model 2: Demand-Based")

# Plotting Model 3 price
p.line(df_sample['Timestamp'], df_sample['CompetitivePrice'],
       line_width=2, color=Category10[10][3], legend_label="Model 3: Competitive")

hover = HoverTool(tooltips=[("Time", "@x{%F %H:%M}"), ("Price", "@y{$0.00}")],
                  formatters={"@x": "datetime"},
                  mode="vline")
p.add_tools(hover)

p.title.text_font_size = '16pt'
p.legend.location = "top_left"
p.legend.click_policy = "hide"

show(p)

# **Pathway Streaming**

In [38]:
!pip install pathway

In [39]:
import pathway as pw

# STEP 1: Defining schema (column types)
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float
    Capacity: int
    Occupancy: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str

# STEP 2: Reading the CSV
data = pw.io.csv.read(
    "sample_100.csv",
    schema=ParkingSchema,
    mode="static"
)


In [40]:
# STEP 3: Encode traffic & vehicle
@pw.udf
def encode_traffic(t): return {"low": 0, "medium": 0.5, "high": 1}.get(t, 0)

@pw.udf
def encode_vehicle(v): return {"car": 1.0, "bike": 0.6, "truck": 1.5}.get(v, 1.0)

data = data.with_columns(
    TrafficScore = encode_traffic(data.TrafficConditionNearby),
    VehicleScore = encode_vehicle(data.VehicleType),
    OccRatio = data.Occupancy / data.Capacity
)

# STEP 4: Calculating raw demand score (Model 2)
@pw.udf
def compute_demand(occ, queue, special, traffic, vehicle):
    return 3*occ + 0.5*queue + 1.5*special + 1.2*traffic + 1.0*vehicle

# STEP 5: Normalising demand score
@pw.udf
def normalize(d): return (d - 2) / (15 - 2)

@pw.udf
def final_price(norm): return min(max(10 * (1 + 0.5 * norm), 5), 20)

# RawDemand
data = data.with_columns(
    RawDemand = compute_demand(
        data.OccRatio,
        data.QueueLength,
        data.IsSpecialDay,
        data.TrafficScore,
        data.VehicleScore
    )
)
# Adding NormDemand and DemandPrice
data = data.with_columns(
    NormDemand = normalize(data.RawDemand),
    DemandPrice = final_price(normalize(data.RawDemand))
)



In [41]:
# Final output
pw.io.jsonlines.write(
    data.select(
        data.SystemCodeNumber,
        data.Timestamp,
        data.Occupancy,
        data.Capacity,
        data.DemandPrice
    ),
    "model2_output.jsonl"
)



/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [42]:
pw.run()

Output()

In [43]:
!head model2_output.jsonl

In [44]:
with open("pathway_model2.py", "w") as f:
    f.write("""import pathway as pw

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Latitude: float
    Longitude: float
    Capacity: int
    Occupancy: int
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str

data = pw.io.csv.read(
    "sample_100.csv",
    schema=ParkingSchema,
    mode="static"
)
@pw.udf
def encode_traffic(t): return {"low": 0, "medium": 0.5, "high": 1}.get(t, 0)

@pw.udf
def encode_vehicle(v): return {"car": 1.0, "bike": 0.6, "truck": 1.5}.get(v, 1.0)

data = data.with_columns(
    TrafficScore = encode_traffic(data.TrafficConditionNearby),
    VehicleScore = encode_vehicle(data.VehicleType),
    OccRatio = data.Occupancy / data.Capacity
)

@pw.udf
def compute_demand(occ, queue, special, traffic, vehicle):
    return 3*occ + 0.5*queue + 1.5*special + 1.2*traffic + 1.0*vehicle

@pw.udf
def normalize(d): return (d - 2) / (15 - 2)

@pw.udf
def final_price(norm): return min(max(10 * (1 + 0.5 * norm), 5), 20)

# Step 1: Add RawDemand separately
data = data.with_columns(
    RawDemand = compute_demand(
        data.OccRatio,
        data.QueueLength,
        data.IsSpecialDay,
        data.TrafficScore,
        data.VehicleScore
    )
)
# Step 2: Now add NormDemand and DemandPrice
data = data.with_columns(
    NormDemand = normalize(data.RawDemand),
    DemandPrice = final_price(normalize(data.RawDemand))
)
pw.io.jsonlines.write(
    data.select(
        data.SystemCodeNumber,
        data.Timestamp,
        data.Occupancy,
        data.Capacity,
        data.DemandPrice
    ),
    "model2_output.jsonl"
)

pw.run()
""")